Try conditioning with [pysheds](https://github.com/ianhorn/pysheds/blob/master/docs/dem-conditioning.md)

In [1]:
import os
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from pysheds.grid import Grid
import seaborn as sns

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
# set the working directory
working_directory = 'data/temp_pysheds'
os.chdir(working_directory)
os.getcwd()

'/Users/ianhorn/Documents/CodeKy/pathways/data-analysis/codeky-da-capstone/project/data/temp_pysheds'

In [3]:
# create a grid from raster
dem = "dem.tif"
grid = Grid.from_raster(dem, nodata=-9999)
dem = grid.read_raster(dem, nodata=-9999)

RasterioIOError: dem.tif: No such file or directory

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
fig.patch.set_alpha(0)

plt.imshow(dem, extent=grid.extent, cmap='terrain', zorder=1)
plt.colorbar(label='Elevation (m)')
plt.grid(zorder=0)
plt.title('Digital elevation map', size=14)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.tight_layout()

In [ ]:
pits = grid.detect_pits(dem)

In [ ]:
# Plot pits
fig, ax = plt.subplots(figsize=(8,6))
fig.patch.set_alpha(0)

plt.imshow(pits, cmap='Greys_r', zorder=1)
plt.title('Pits', size=14)
plt.tight_layout()

In [ ]:
pit_filled_dem =grid.fill_pits(dem)
pits = grid.detect_pits(pit_filled_dem)
assert not pits.any()

In [ ]:
depressions = grid.detect_depressions(pit_filled_dem)

In [ ]:
# Plot depressions
fig, ax = plt.subplots(figsize=(8,6))
fig.patch.set_alpha(0)

plt.imshow(depressions, cmap='Greys_r', zorder=1)
plt.title('Depressions', size=14)
plt.tight_layout()

In [ ]:
flooded_dem = grid.fill_depressions(pit_filled_dem)
depressions = grid.detect_depressions(flooded_dem)
assert not depressions.any()

In [ ]:
flats = grid.detect_flats(flooded_dem)

In [ ]:
# Plot flats
fig, ax = plt.subplots(figsize=(8,6))
fig.patch.set_alpha(0)

plt.imshow(flats, cmap='Greys_r', zorder=1)
plt.title('Flats', size=14)
plt.tight_layout()

In [ ]:
inflated_dem = grid.resolve_flats(flooded_dem)
flats = grid.detect_flats(inflated_dem)
assert not flats.any()

In [ ]:
import numpy as np

# Detect flats again after resolve_flats()
flats = grid.detect_flats(inflated_dem)

# Print summary
print("Number of flat cells:", np.count_nonzero(flats))

# Optionally visualize
import matplotlib.pyplot as plt
plt.imshow(flats, cmap='gray')
plt.title("Remaining Flats After resolve_flats()")
plt.colorbar()
plt.show()

In [ ]:
# Compute flow direction based on corrected DEM
fdir = grid.flowdir(inflated_dem)

# Compute flow accumulation based on computed flow direction
acc = grid.accumulation(fdir)


In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
fig.patch.set_alpha(0)
im = ax.imshow(acc, zorder=2,
               cmap='cubehelix',
               norm=colors.LogNorm(1, acc.max()),
               interpolation='bilinear')
plt.colorbar(im, ax=ax, label='Upstream Cells')
plt.title('Flow Accumulation', size=14)
plt.tight_layout()